
# 📊 Ликвидность фьючерсов MOEX (FORTS) — загрузка через ISS

Этот ноутбук помогает получить **дневную ликвидность** по фьючерсам Московской биржи (рынок **FORTS**) из публичного API **ISS**:
- эндпоинт: `/iss/history/engines/futures/markets/forts/securities`  
- таблица ответа: `history`

Вы можете выгрузить данные:
- **за один день** (`date=YYYY-MM-DD`), **или**
- **за диапазон дат** (`from=YYYY-MM-DD` и `till=YYYY-MM-DD`).

Далее ноутбук агрегирует метрики по каждому фьючерсу (`SECID`) и показывает **топ‑N** по выбранному показателю.

---

## Что именно мы считаем

Полезные поля из `history`:
- `VOLUME` — объём в контрактах за день,
- `VALUE` — оборот в рублях за день,
- `NUMTRADES` — число сделок за день,
- также доступны `OPEN`, `HIGH`, `LOW`, `CLOSE` и другие поля.
  
После загрузки за период мы суммируем `VALUE/VOLUME/NUMTRADES` по каждому `SECID` и ранжируем.

> 🔎 Если нужна **онлайн‑ликвидность в течение дня**, используйте эндпоинт
> `/iss/engines/futures/markets/forts/securities?iss.only=marketdata` (поля накапливаются в течение дня).
> Здесь мы концентрируемся на **дневной истории**.



## ⚙️ Зависимости

Если необходимо, установите зависимости:


In [ ]:

# Установите зависимости (раскомментируйте при необходимости)
# %pip install requests pandas



## 🧩 Базовые настройки и вспомогательные функции
- Аккуратные GET‑запросы с ретраями
- Постраничная загрузка (`start`, `limit`)
- Преобразование секций ISS в DataFrame


In [1]:

import time
from typing import Dict, Any, List, Optional

import requests
import pandas as pd

BASE = "https://iss.moex.com"
HIST_PATH = "/iss/history/engines/futures/markets/forts/securities.json"
HEADERS = {"User-Agent": "Mozilla/5.0 (moex-forts-liquidity-notebook)"}

PAGE_LIMIT_DEFAULT = 100  # размер страницы (можно менять)
RETRIES = 10
SLEEP_BETWEEN_REQUESTS = 0.2  # в секундах
RETRY_SLEEP = 1.0

def _get_json(path: str, params: Dict[str, Any]) -> Dict[str, Any]:
    """GET c несколькими попытками (ретраи)."""
    url = BASE + path
    last_exc = None
    for _ in range(RETRIES):
        try:
            r = requests.get(url, params=params, headers=HEADERS, timeout=30)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_exc = e
            time.sleep(RETRY_SLEEP)
    raise RuntimeError(f"ISS request failed after {RETRIES} retries: {url} params={params}\n{last_exc}")

def _section_to_df(js: Dict[str, Any], key: str) -> pd.DataFrame:
    """Секция формата {'columns':[...], 'data':[[...],...]} -> DataFrame."""
    sec = js.get(key, {})
    if not sec or not sec.get("data"):
        return pd.DataFrame()
    return pd.DataFrame(sec["data"], columns=sec["columns"])    



## 📥 Загрузка истории FORTS (один день или диапазон дат)

Функции ниже:
- `fetch_forts_history_for_date(date)`: одна дата;
- `fetch_forts_history_range(from_date, till_date)`: диапазон дат;
- `fetch_forts_history(date=None, from_date=None, till_date=None)`: универсальный вход.


In [2]:

def fetch_forts_history_for_date(date: str,
                                 page_limit: int = PAGE_LIMIT_DEFAULT,
                                 sleep_sec: float = SLEEP_BETWEEN_REQUESTS) -> pd.DataFrame:
    """Загрузить историю FORTS за одну дату (таблица 'history')."""
    frames: List[pd.DataFrame] = []
    start = 0
    while True:
        params = {
            "date": date,
            "start": start,
            "limit": page_limit,
        }
        js = _get_json(HIST_PATH, params)
        df = _section_to_df(js, "history")
        if df.empty:
            break
        # нормализуем имена колонок (ISS обычно CAPS, но на всякий случай):
        df.columns = [c.upper() for c in df.columns]
        frames.append(df)

        if len(df) < page_limit:
            break
        start += page_limit
        time.sleep(sleep_sec)

    if not frames:
        return pd.DataFrame()
    out = pd.concat(frames, ignore_index=True)
    # приведение типов
    for col in ("VALUE", "VOLUME", "NUMTRADES"):
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors="coerce").fillna(0)
    if "TRADEDATE" in out.columns:
        out["TRADEDATE"] = pd.to_datetime(out["TRADEDATE"], errors="coerce").dt.date
    return out

def fetch_forts_history_range(from_date: str, till_date: str,
                              page_limit: int = PAGE_LIMIT_DEFAULT) -> pd.DataFrame:
    """Загрузить историю за диапазон дат (включительно)."""
    d0 = pd.to_datetime(from_date).date()
    d1 = pd.to_datetime(till_date).date()
    if d1 < d0:
        raise ValueError("till_date < from_date")

    frames: List[pd.DataFrame] = []
    cur = d0
    while cur <= d1:
        df = fetch_forts_history_for_date(cur.isoformat(), page_limit=page_limit)
        if not df.empty:
            frames.append(df)
        cur += pd.Timedelta(days=1)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def fetch_forts_history(date: Optional[str] = None,
                        from_date: Optional[str] = None,
                        till_date: Optional[str] = None,
                        page_limit: int = PAGE_LIMIT_DEFAULT) -> pd.DataFrame:
    """Универсальная обёртка: либо date, либо (from_date, till_date)."""
    if date and (from_date or till_date):
        raise ValueError("Укажите либо 'date', либо 'from_date'+'till_date', но не вместе.")
    if date:
        return fetch_forts_history_for_date(date, page_limit=page_limit)
    if from_date and till_date:
        return fetch_forts_history_range(from_date, till_date, page_limit=page_limit)
    # по умолчанию — сегодня
    today = pd.Timestamp.today().date().isoformat()
    return fetch_forts_history_for_date(today, page_limit=page_limit)



## 🧮 Агрегация по фьючерсам и ранжирование

Суммируем выбранные метрики (`VALUE`, `VOLUME`, `NUMTRADES`) по каждому `SECID` и выдаём топ‑N.


In [3]:

def aggregate_and_rank(df_history: pd.DataFrame,
                       sort_by: str = "VALUE",
                       top_n: int = 20) -> pd.DataFrame:
    """Агрегируем по SECID и ранжируем по выбранной метрике."""
    if df_history.empty:
        return pd.DataFrame(columns=["SECID", "VALUE", "VOLUME", "NUMTRADES"])

    required = {"SECID", "VALUE", "VOLUME", "NUMTRADES"}
    missing = required - set(df_history.columns)
    if missing:
        raise ValueError(f"В данных нет колонок: {missing}")

    agg = (
        df_history.groupby("SECID", as_index=False)[["VALUE", "VOLUME", "NUMTRADES"]]
        .sum()
        .sort_values(sort_by, ascending=False)
        .head(top_n)
        .reset_index(drop=True)
    )
    return agg



## 🕹️ Параметры запроса и запуск

Выберите один из вариантов:
- Указать **одну дату**: `DATE = "YYYY-MM-DD"` и оставить `FROM_DATE=None`, `TILL_DATE=None`  
- Указать **диапазон дат**: `FROM_DATE = "YYYY-MM-DD"`, `TILL_DATE = "YYYY-MM-DD"` и оставить `DATE=None`

Параметры:
- `SORT_BY`: по какой метрике ранжировать (`"VALUE"`, `"VOLUME"`, `"NUMTRADES"`)
- `TOP_N`: размер топа
- `SAVE_CSV`, `SAVE_EXCEL`: пути для сохранения (или `None`, чтобы не сохранять)


In [4]:

# Пример настроек — измените под задачу
DATE = None                # например: "2025-09-22" или None
FROM_DATE = "2025-09-15"   # например: "2025-09-15" или None
TILL_DATE = "2025-09-22"   # например: "2025-09-22" или None

SORT_BY = "VALUE"          # "VALUE" | "VOLUME" | "NUMTRADES"
TOP_N = 100000

SAVE_CSV = None            # например: "forts_liquidity_top20.csv" или None
SAVE_EXCEL = 'futures.xlsx'          # например: "forts_liquidity_top20.xlsx" или None



## ▶️ Запуск загрузки и формирование топа


In [ ]:
history_df.to_excel('history.xlsx', index=False)

In [5]:

# Загружаем историю
history_df = fetch_forts_history(date=DATE, from_date=FROM_DATE, till_date=TILL_DATE)
print(f"Строк истории получено: {len(history_df)}")
display(history_df.head(10))

# Ранжируем
top_df = aggregate_and_rank(history_df, sort_by=SORT_BY, top_n=TOP_N)
print(f"\nTOP {TOP_N} по {SORT_BY} (сумма за выбранный период):\n")
display(top_df)

# Сохранение по желанию
if SAVE_CSV:
    top_df.to_csv(SAVE_CSV, index=False)
    print(f"\nСохранено в CSV: {SAVE_CSV}")
if SAVE_EXCEL:
    top_df.to_excel(SAVE_EXCEL, index=False)
    print(f"Сохранено в Excel: {SAVE_EXCEL}")


Строк истории получено: 4826


,BOARDID,TRADEDATE,SECID,OPEN,LOW,HIGH,CLOSE,OPENPOSITIONVALUE,VALUE,VOLUME,OPENPOSITION,SETTLEPRICE,SWAPRATE,WAPRICE,SETTLEPRICEDAY,CHANGE,QTY,NUMTRADES
0,RFUD,2025-09-15,AEH6,NaN,NaN,NaN,NaN,1562550.0,0.0,0.0,66,23.675,0.0,NaN,23.646,NaN,NaN,0
1,RFUD,2025-09-15,AEM6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,24.230,0.0,NaN,24.201,NaN,NaN,0
2,RFUD,2025-09-15,AEU5,23.322,22.75,23.322,22.75,5147100.0,1024431.0,45.0,228,22.575,0.0,22.765,22.548,-0.84,42.0,4
3,RFUD,2025-09-15,AEU5AEZ5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0.701,0.0,NaN,0.701,NaN,NaN,0
4,RFUD,2025-09-15,AEZ5,23.491,23.35,23.491,23.35,6750040.0,868978.0,37.0,290,23.276,0.0,23.486,23.249,-1.49,1.0,4
5,RFUD,2025-09-15,AEZ5AEH6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0.399,0.0,NaN,0.397,NaN,NaN,0
6,RFUD,2025-09-15,AFH6,6520.000,6520.00,6520.000,6520.00,26072.0,6520.0,1.0,4,6518.000,0.0,6520.000,6520.000,-3.39,1.0,1
7,RFUD,2025-09-15,AFKS_CLT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,15.200,NaN,NaN,NaN,NaN,NaN,0
8,RFUD,2025-09-15,AFLT_CLT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,60.050,NaN,NaN,NaN,NaN,NaN,0
9,RFUD,2025-09-15,AFU5,6029.000,5948.00,6070.000,6013.00,819559660.0,102069383.0,16992.0,136366,6010.000,0.0,6007.000,5993.000,-0.41,4.0,3224



TOP 100000 по VALUE (сумма за выбранный период):



,SECID,VALUE,VOLUME,NUMTRADES
0,SiZ5,3.960564e+11,4587363.0,868051
1,CRZ5,3.388580e+11,27869180.0,666890
2,CRU5,3.213468e+11,27602596.0,572134
3,SiU5,2.975028e+11,3582997.0,662667
4,GDZ5,2.824864e+11,922777.0,291404
...,...,...,...,...
884,W4U6,0.000000e+00,0.0,0
885,W4V5W4X5,0.000000e+00,0.0,0
886,W4X5W4Z5,0.000000e+00,0.0,0
887,WUSH_CLT,0.000000e+00,0.0,0


Сохранено в Excel: futures.xlsx



---

## ℹ️ Полезные замечания

- Если нужна **онлайн‑ликвидность в течение дня** (нарастающим итогом с начала текущей сессии), используйте:
  `/iss/engines/futures/markets/forts/securities?iss.only=marketdata` и аналогично агрегируйте поля `VOLUME`, `VALTODAY`, `NUMTRADES`.
- Для **классификации фьючерсов по базовому активу** (акция/валюта/индекс/товар) удобно брать справочники:
  - `/iss/statistics/engines/futures/markets/forts/series` (поля с базовым активом, например `ASSETCODE`),
  - `/iss/referencedata/engines/futures/markets/forts/securities` (Reference Data 2.0).
- Если потребуется: добавьте фильтр по `SECID`/маскам (`like 'Si%'`, и т.п.) уже после загрузки `history_df`.
